In [1]:
from n2v.models import N2V
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.image import imread, imsave
from csbdeep.io import save_tiff_imagej_compatible
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator

2024-04-15 09:51:54.122358: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 09:51:54.199643: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 09:51:54.489746: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 09:51:55.230471: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

dome_path = './../../ppujol/test_n2v_linux/' # Fill with the path of the dome
name_of_dome = '21.tif' # Fill with the name of the dome
save_dome_path = './../../ppujol/test_n2v_linux/' # Fill with the path to save the dome
name_save_dome = '21_denoised.tif'

In [3]:
datagen = N2V_DataGenerator()
img = datagen.load_imgs_from_directory(directory=dome_path, filter=name_of_dome, dims='TZXY')

print(img[0].shape)

(9, 380, 576, 576, 1)


In [10]:
model_name = '2d_images_from_stack'
basedir = './n2v_2d_black_line/models_test/'
model = N2V(config=None, name=model_name, basedir=basedir)

Loading network weights from 'weights_best.h5'.


In [74]:
def denoise(img):

    d3_pred = []

    for image in img:
        
        image = image[:, :, 0]
        # print(image.shape)

        non_zero_cols = np.any(image != 0, axis = 0)
        first_non_zero_col = np.argmax(non_zero_cols)
        last_non_zero_col = len(non_zero_cols) - np.argmax(non_zero_cols[::-1]) - 1
        clean_image = image[:, first_non_zero_col:last_non_zero_col + 1]

        pred = model.predict(clean_image, axes='YX')

        full_pred = np.zeros_like(image)

        full_pred[:, first_non_zero_col:last_non_zero_col + 1] = pred
        
        d3_pred.append(full_pred)
    
    return d3_pred

In [75]:
frame = -1 # Write the frame number or -1 if you want to denoise all of the stack

stack = []

if frame == -1:
    for im in img[0]:
        # print(im.shape)
        stack.append(denoise(im))

elif frame < img[0].shape[0] and frame > -1:
    for im in img[0][frame]:
        stack.append(denoise(img))
elif frame > img[0].shape[0] or frame <= 0:
    print('no frame')



1/1 [==============================] - 1s 1s/step


In [85]:
stack = np.array(stack)

(9, 380, 576, 576)
9 380 576 576


In [91]:
save_tiff_imagej_compatible(str(save_dome_path)+str(name_save_dome), stack, axes='TZXY')